# IMPORT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch

In [ ]:
pip install mediapy

In [ ]:
pip install legacy

In [5]:
from __future__ import print_function
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
import os
import sys
import torch
from PIL import Image
import matplotlib.pyplot as plt
import pickle
import PIL.Image
from time import perf_counter
import torchvision.transforms as transforms
import torchvision.models as models

import copy

In [6]:
sys.path.insert(0, '/content/stylegan2-ada-pytorch')
import dnnlib
sys.path.insert(1, '/content/stylegan2-ada-pytorch')
import torch_utils.persistence
sys.path.insert(1, '/content/stylegan2-ada-pytorch')
import style_mixing

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load GAN

In [8]:
import warnings
warnings.filterwarnings('ignore')
sys.stdout = open(os.devnull, 'w')

In [9]:
with open('/content/drive/MyDrive/nni_ex4/wikiart256-fid689.pkl', 'rb') as f:
    G = pickle.load(f)['G_ema'].cuda()  # torch.nn.Module
z = torch.randn([1, G.z_dim]).cuda()    # latent codes
c = None                                # class labels (not used in this example)
img = G(z, c)                           # NCHW, float32, dynamic range [-1, +1]

# Loss Functions


# Content Loss

In [10]:
class ContentLoss(nn.Module):
    def __init__(self, target,):
        super(ContentLoss, self).__init__()
        self.target = target.detach()

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

# Style Loss


In [11]:
def gram_matrix(input):
    a, b, c, d = input.size()
    features = input.view(a * b, c * d) 
    G = torch.mm(features, features.t())
    return G.div(a * b * c * d)

In [12]:
class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input

# Style Mixing

In [13]:
def style_mixing(G,w_content,w_style,styles_lvl):
    w = w_content.clone()
    w[styles_lvl] = w_style[styles_lvl]
    image = G.synthesis(w[np.newaxis], noise_mode='const')
    image = (image.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    return image[0].cpu().numpy()

In [ ]:
content_seeds = [31, 15, 83]
style_seeds = [47, 139, 89]
content_z = np.stack([np.random.RandomState(seed).randn(G.z_dim) for seed in content_seeds])
content_w = G.mapping(torch.from_numpy(content_z).to(device), None)
content_images = G.synthesis(content_w, noise_mode="const")
content_images=(content_images.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
style_z = np.stack([np.random.RandomState(seed).randn(G.z_dim) for seed in style_seeds])
style_w = G.mapping(torch.from_numpy(style_z).to(device), None)
style_images = G.synthesis(style_w, noise_mode="const")
style_images=(style_images.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8).cpu().numpy()
for i in range(len(content_seeds)):
    content=content_images[i]
    style=style_images[i]
    plt.imshow(content)
    plt.title("content")
    plt.show()
    plt.imshow(style)
    plt.title("style")
    plt.show()
    mix=style_mixing(G,content_w[i],style_w[i],range(7,14))
    plt.imshow(mix)
    plt.title("mix")
    plt.show()
  

In [62]:
for i in range(len(all_seeds)):
  PIL.Image.fromarray(all_images[i], 'RGB').save(f"{all_seeds[i]}.png")

# Importing the Model


In [63]:
cnn = models.vgg19(pretrained=True).features.to(device).eval()

In [ ]:
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

# create a module to normalize input image so we can easily put it in a
# nn.Sequential
class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        # .view the mean and std to make them [C x 1 x 1] so that they can
        # directly work with image Tensor of shape [B x C x H x W].
        # B is batch size. C is number of channels. H is height and W is width.
        self.mean = torch.tensor(mean).view(-1, 1, 1)
        self.std = torch.tensor(std).view(-1, 1, 1)

    def forward(self, img):
        # normalize img
        return (img - self.mean) / self.std

In [ ]:
content_layers_default = ['conv_4']
style_layers_default = ['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']

def get_style_model_and_losses(cnn, normalization_mean, normalization_std,
                               style_img, content_img,
                               content_layers=content_layers_default,
                               style_layers=style_layers_default):
    normalization = Normalization(normalization_mean, normalization_std).to(device)

    content_losses = []
    style_losses = []

    model = nn.Sequential(normalization)

    i = 0  
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        if name in content_layers:
            target = model(content_img).detach()
            content_loss = ContentLoss(target)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        if name in style_layers:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]

    return model, style_losses, content_losses



```
# This is formatted as code
```

# Optimizing on W+


In [99]:
def get_input_optimizer(input_img):
    # this line to show that input is a parameter that requires a gradient
    optimizer = optim.Adam([input_img])
    return optimizer

In [98]:
def run_style_transfer(cnn, normalization_mean, normalization_std,
                       content_img, style_img, input_img,content_seed,a, num_steps=300,
                       style_weight=1000000, content_weight=1):
    model, style_losses, content_losses = get_style_model_and_losses(cnn,
        normalization_mean, normalization_std, style_img, content_img)
    input_img.requires_grad_(True)
    model.requires_grad_(False)
    all_seeds=[content_seed]
    all_z = np.stack([np.random.RandomState(seed).randn(G.z_dim) for seed in all_seeds])
    all_w = G.mapping(torch.from_numpy(all_z).to(device), None)
    input_img = G.synthesis(all_w, noise_mode="const")
    all_w = torch.tensor(all_w, dtype=torch.float32, device=device, requires_grad=True) # pylint: disable=not-callable
    optimizer = get_input_optimizer(all_w)
    run = [0]
    for i in range(num_steps):
        input_img = (G.synthesis(all_w, noise_mode="const")+1)/2
        def closure():
            with torch.no_grad():
                input_img.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            style_score = 0
            content_score = 0

            for sl in style_losses:
                style_score += sl.loss
            for cl in content_losses:
                content_score += cl.loss

            style_score *= style_weight
            content_score *= content_weight

            loss = a * style_score + content_score
            loss.backward(retain_graph=True)
            return   a * style_score + content_score

        optimizer.step(closure)
    with torch.no_grad():
        input_img.clamp_(0, 1)

    return input_img

# Loading the Images


In [100]:
imsize = 256 
loader = transforms.Compose([
    transforms.Resize(imsize),  
    transforms.ToTensor()])  

def image_loader(image_name):
    image = Image.open(image_name)
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float)

In [101]:

plt.ion()
def imshow(tensor, title=None):
    image = tensor.cpu().clone() 
    image = image.squeeze(0)      
    image = transforms.ToPILImage()(image)
    plt.imshow(image)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) 


# **RUN Optimization W+**

In [ ]:
content_seeds = [31, 15, 83]
style_seeds = [47, 139, 89]
a=[0.1,0.05,0.1]
for i in range(len(content_seeds)):
  style_img = image_loader(f"/content/{style_seeds[i]}.png")
  content_img = image_loader(f"/content/{content_seeds[i]}.png")
  output = run_style_transfer(cnn, cnn_normalization_mean, cnn_normalization_std,
                            content_img, style_img, input_img,content_seeds[i],a[i])
  plt.figure()
  imshow(style_img, title='Style Image')
  plt.figure()
  imshow(content_img, title='Content Image')
  imshow(output,title='mix')
  plt.ioff()
  plt.show()